In [ ]:
# train
nohup python sr.py -p train -c config/tomo_sr_sr3_16_128_v2.json -enable_wandb -log_wandb_ckpt &

In [ ]:
# infer
python infer.py -p val -c config/tomo_sr_sr3_16_128_v2.json -enable_wandb -gpu 0 

## Создание датасета

In [2]:
import pandas as pd

df = pd.read_csv('/home/d_korostelev/Projects/super_resolution/data/v1_dataset_DeepRockSR.csv')
df

,path,material,split
0,/home/d_korostelev/Projects/super_resolution/d...,sandstone,train
1,/home/d_korostelev/Projects/super_resolution/d...,sandstone,train
2,/home/d_korostelev/Projects/super_resolution/d...,sandstone,train
3,/home/d_korostelev/Projects/super_resolution/d...,sandstone,train
4,/home/d_korostelev/Projects/super_resolution/d...,sandstone,train
...,...,...,...
23995,/home/d_korostelev/Projects/super_resolution/d...,shuffled,valid
23996,/home/d_korostelev/Projects/super_resolution/d...,shuffled,valid
23997,/home/d_korostelev/Projects/super_resolution/d...,shuffled,valid
23998,/home/d_korostelev/Projects/super_resolution/d...,shuffled,valid


# структура проекта
dataset/tomo_train/
├── hr_128 # it's same with sr_16_128 directory if you don't have ground-truth images.
├── lr_16 # vinilla low resolution images
└── sr_16_128 # images ready to super resolution

dataset/tomo_val/
├── hr_128 # it's same with sr_16_128 directory if you don't have ground-truth images.
├── lr_16 # vinilla low resolution images
└── sr_16_128 # images ready to super resolution

In [17]:
from tqdm import tqdm
import os
from pathlib import Path
import cv2

def create_sr_image_dataset(df, out_path, orig_size=512, resize_factor=4):
        
    lr_size = orig_size//resize_factor
    orig_folder = os.path.join(out_path, f'hr_{orig_size}')
    lr_folder = os.path.join(out_path, f'lr_{lr_size}')
    sr_folder = os.path.join(out_path, f'sr_{lr_size}_{orig_size}')
     
    
    for p in [out_path, orig_folder, lr_folder, sr_folder]:
        Path(p).mkdir(exist_ok=True, parents=True)

    
    for i, row in tqdm(df.iterrows()):
        orig_image = cv2.imread(row['path'])
        sr_image = cv2.resize(orig_image, (orig_size,orig_size))
        lr_image = cv2.resize(orig_image, (lr_size,lr_size))
        
        img_name = os.path.basename(row['path'])
        
        # print(orig_folder)
        cv2.imwrite(os.path.join(orig_folder, img_name), sr_image)
        cv2.imwrite(os.path.join(sr_folder, img_name), sr_image)
        cv2.imwrite(os.path.join(lr_folder, img_name), lr_image) 
        


df_val = df[df.split == 'test']
val_dataset_path = '/home/d_korostelev/Projects/super_resolution/sr_src_original/dataset/tomo/val'
!rm -rf {val_dataset_path}
create_sr_image_dataset(df_val, out_path=val_dataset_path)

2400it [00:30, 79.11it/s]


In [15]:
!rm -rf {val_dataset_path}

In [14]:
df_train = df[df.split != 'test']
train_dataset_path = '/home/d_korostelev/Projects/super_resolution/sr_src_original/dataset/tomo/train'
!rm -rf {train_dataset_path}
create_sr_image_dataset(df_train, out_path=train_dataset_path)

21600it [10:47, 33.38it/s]


In [19]:
!nvidia-smi

Wed Sep 21 17:50:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:61:00.0 Off |                    0 |
| N/A   40C    P0    37W / 250W |  13289MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  On   | 00000000:DB:00.0 Off |                    0 |
| N/A   

In [21]:
from random import randrange
randrange(0, 12)

8

In [ ]:
# Создание альтернативного датасета (тут только HR картинки - LR картинки в динамике через ресайз

In [22]:
from tqdm import tqdm
import os
from pathlib import Path
import cv2

def create_sr_image_dataset(df, out_path, orig_size=256, resize_factor=4):
        
    orig_folder = os.path.join(out_path, f'hr_{orig_size}')     
    
    for p in [out_path, orig_folder]:
        Path(p).mkdir(exist_ok=True, parents=True)

    for i, row in tqdm(df.iterrows()):
        orig_image = cv2.imread(row['path'])
        
        img_name = os.path.basename(row['path'])
        cv2.imwrite(os.path.join(orig_folder, img_name), orig_image)
        

df_val = df[df.split == 'test']
val_dataset_path = '/home/d_korostelev/Projects/super_resolution/sr_src_original/dataset/tomo_hr/val'
!rm -rf {val_dataset_path}
create_sr_image_dataset(df_val, out_path=val_dataset_path)

2400it [01:13, 32.71it/s]


In [24]:
df_train = df[df.split != 'test']
train_dataset_path = '/home/d_korostelev/Projects/super_resolution/sr_src_original/dataset/tomo_hr/train'
!rm -rf {train_dataset_path}
create_sr_image_dataset(df_train, out_path=train_dataset_path)

21600it [12:23, 29.07it/s]


In [25]:
!nvidia-smi

Wed Sep 21 22:19:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:61:00.0 Off |                    0 |
| N/A   45C    P0    38W / 250W |   1482MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  On   | 00000000:DB:00.0 Off |                    0 |
| N/A   

# Посмотрим на Perseptual loss 

In [17]:
import lpips
# loss_fn_alex = lpips.LPIPS(net='alex') # best forward scores
loss_fn_vgg = lpips.LPIPS(net='vgg') # closer to "traditional" perceptual loss, when used for optimization

import torch
img0 = torch.zeros(1,1,64,64) # image should be RGB, IMPORTANT: normalized to [-1,1]
img1 = torch.ones(1,1,64,64)
d = loss_fn_vgg(img0, img1)
d

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: /home/d_korostelev/env_torch/lib/python3.7/site-packages/lpips/weights/v0.1/vgg.pth


tensor([[[[0.4945]]]], grad_fn=<AddBackward0>)

In [ ]:
d

In [12]:
sh = torch.Tensor([.458,.448,.450])[None,:,None,None]
print(sh.shape)
sh

torch.Size([1, 3, 1, 1])


tensor([[[[0.4580]],

         [[0.4480]],

         [[0.4500]]]])

In [18]:
img0_sh = img0 - sh


In [21]:
img0_sh[0, 2, ...]

tensor([[-0.4500, -0.4500, -0.4500,  ..., -0.4500, -0.4500, -0.4500],
        [-0.4500, -0.4500, -0.4500,  ..., -0.4500, -0.4500, -0.4500],
        [-0.4500, -0.4500, -0.4500,  ..., -0.4500, -0.4500, -0.4500],
        ...,
        [-0.4500, -0.4500, -0.4500,  ..., -0.4500, -0.4500, -0.4500],
        [-0.4500, -0.4500, -0.4500,  ..., -0.4500, -0.4500, -0.4500],
        [-0.4500, -0.4500, -0.4500,  ..., -0.4500, -0.4500, -0.4500]])

In [30]:
import numpy as np
fake_img = np.ones((12, 12))
sr_img = np.ones((12, 12))
hr_img = np.ones((12, 12))

np.transpose(np.concatenate((fake_img, sr_img, hr_img), axis=1), [2, 0, 1])

ValueError: axes don't match array